In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas, math

In [3]:
list_dth = ['2023101602']  #BusinessGroupingAna_FM0601_2023101602utc24hrs.tsv
df_fm0601 = pandas.concat(map(lambda x: pandas.read_csv('data/BusinessGroupingAna_FM0601_{}utc24hrs.tsv'.format(x), sep='\t', index_col=0), list_dth))
df_fm0601 = df_fm0601.drop(['PublisherId', 'exp_split'], axis=1)
df_fm0601 = df_fm0601.groupby('group').sum().reset_index().sort_values('group')
df_fm0601

,group,Nimp,sumPred,sumLabel,sumlogloss
0,1d_carter:0,27185885,125786.311227,125068,546814.155961
1,1d_carter:1,153423,788.383415,794,4200.591901
2,30d_buyer:0,26963871,125035.482905,124368,542808.485201
3,30d_buyer:1,375437,1539.211737,1494,8206.262661
4,4h_viewer:0,26669958,116402.296148,115639,511213.941359
...,...,...,...,...,...
997391,pubid:63_x_pub-9999847103134086,75,0.090133,0,0.090209
997392,with_uv:0,21505115,93234.598245,93330,398222.991704
997393,with_uv:1,5834193,33340.096397,32532,152791.756158
997394,zero_profile_user:0,13487363,69289.799377,68207,295401.706392


In [4]:
import ROOT
from ROOT import TCanvas, TFormula, TF1, TFile, TChain, TTree, TH1D, TH2D, TH1I, TClass, TLegend, TProfile, TPython, TDatime, TH2C, TEfficiency, TGraphAsymmErrors, TMath, TLine
from ROOT import gROOT, gObjectTable, gSystem, gPad
from ROOT import gStyle 
gStyle.SetNumberContours(100)

%jsroot on

Welcome to JupyROOT 6.26/04


In [5]:
def Cal_loglikelihood_binomial(n_pos, mean_pred, n_total):
    return (
        ROOT.Math.lgamma(n_total+1) - ROOT.Math.lgamma(n_pos+1) - ROOT.Math.lgamma(n_total-n_pos+1)
        + n_pos * TMath.Log(mean_pred) + (n_total - n_pos) * ROOT.Math.log1p(-mean_pred)
    )

def Cal_loglikelihood_ratio_binomial(n_pos, mean_pred, n_total):
    log_max_ll = 0.
    if n_pos>0 and (n_total-n_pos)>0:
        log_max_ll = (
            ROOT.Math.lgamma(n_total+1) - ROOT.Math.lgamma(n_pos+1) - ROOT.Math.lgamma(n_total-n_pos+1)
            + n_pos * TMath.Log(float(n_pos)/n_total) + (n_total - n_pos) * ROOT.Math.log1p(-float(n_pos)/n_total)
        )
    return (
        ROOT.Math.lgamma(n_total+1) - ROOT.Math.lgamma(n_pos+1) - ROOT.Math.lgamma(n_total-n_pos+1)
        + n_pos * TMath.Log(mean_pred) + (n_total - n_pos) * ROOT.Math.log1p(-mean_pred)
    ) - log_max_ll

In [7]:
df_fm0601['loglikelihood'] = df_fm0601.apply(lambda row: Cal_loglikelihood_binomial(row['sumLabel'], row['sumPred']/row['Nimp'], row['Nimp']), axis=1)
df_fm0601['loglikelihoodratio'] = df_fm0601.apply(lambda row: Cal_loglikelihood_ratio_binomial(row['sumLabel'], row['sumPred']/row['Nimp'], row['Nimp']), axis=1)

In [8]:
df_fm0601

,group,Nimp,sumPred,sumLabel,sumlogloss,loglikelihood,loglikelihoodratio
0,1d_carter:0,27185885,125786.311227,125068,546814.155961,-8.849372,-2.064431
1,1d_carter:1,153423,788.383415,794,4200.591901,-4.275054,-0.020063
2,30d_buyer:0,26963871,125035.482905,124368,542808.485201,-8.575235,-1.793107
3,30d_buyer:1,375437,1539.211737,1494,8206.262661,-5.244948,-0.673341
4,4h_viewer:0,26669958,116402.296148,115639,511213.941359,-9.264958,-2.519077
...,...,...,...,...,...,...,...
997391,pubid:63_x_pub-9999847103134086,75,0.090133,0,0.090209,-0.090187,-0.090187
997392,with_uv:0,21505115,93234.598245,93330,398222.991704,-6.687719,-0.049006
997393,with_uv:1,5834193,33340.096397,32532,152791.756158,-16.040825,-9.929690
997394,zero_profile_user:0,13487363,69289.799377,68207,295401.706392,-15.030176,-8.548620


In [10]:
val_ll = sum(df_fm0601['loglikelihood'])
val_llr = sum(df_fm0601['loglikelihoodratio'])
n_groups = df_fm0601['group'].count()
prob_ll = math.exp(val_ll/n_groups)
prob_llr = math.exp(val_llr/n_groups)
print(val_ll, val_llr)
print(prob_ll, prob_llr)

-86026.36249438042 -48513.72921389374
0.9173639806702479 0.9525236062857443


In [ ]:
=====

In [12]:
list_dth = ['2023101602']  #BusinessGroupingAna_FM0601_2023101602utc24hrs.tsv
df_fm0601 = pandas.concat(map(lambda x: pandas.read_csv('data/BusinessGroupingAna_FM0601_delay2h_{}utc24hrs.tsv'.format(x), sep='\t', index_col=0), list_dth))
df_fm0601 = df_fm0601.drop(['PublisherId', 'exp_split'], axis=1)
df_fm0601 = df_fm0601.groupby('group').sum().reset_index().sort_values('group')
df_fm0601

,group,Nimp,sumPred,sumLabel,sumlogloss
0,1d_carter:0,27185885,126773.445387,125068,556662.512890
1,1d_carter:1,153423,791.492762,794,4206.374071
2,30d_buyer:0,26963871,126017.297175,124368,552632.645543
3,30d_buyer:1,375437,1547.640974,1494,8236.241417
4,4h_viewer:0,26669958,117399.470002,115639,520968.463595
...,...,...,...,...,...
997391,pubid:63_x_pub-9999847103134086,75,0.091443,0,0.091520
997392,with_uv:0,21505115,93655.606007,93330,407697.813471
997393,with_uv:1,5834193,33909.332142,32532,153171.073490
997394,zero_profile_user:0,13487363,69894.824760,68207,299608.316706


In [13]:
df_fm0601['loglikelihood'] = df_fm0601.apply(lambda row: Cal_loglikelihood_binomial(row['sumLabel'], row['sumPred']/row['Nimp'], row['Nimp']), axis=1)
df_fm0601['loglikelihoodratio'] = df_fm0601.apply(lambda row: Cal_loglikelihood_ratio_binomial(row['sumLabel'], row['sumPred']/row['Nimp'], row['Nimp']), axis=1)

In [14]:
df_fm0601

,group,Nimp,sumPred,sumLabel,sumlogloss,loglikelihood,loglikelihoodratio
0,1d_carter:0,27185885,126773.445387,125068,556662.512890,-18.361897,-11.576957
1,1d_carter:1,153423,791.492762,794,4206.374071,-4.258978,-0.003988
2,30d_buyer:0,26963871,126017.297175,124368,552632.645543,-17.673089,-10.890961
3,30d_buyer:1,375437,1547.640974,1494,8236.241417,-5.515978,-0.944371
4,4h_viewer:0,26669958,117399.470002,115639,520968.463595,-20.070329,-13.324448
...,...,...,...,...,...,...,...
997391,pubid:63_x_pub-9999847103134086,75,0.091443,0,0.091520,-0.091499,-0.091499
997392,with_uv:0,21505115,93655.606007,93330,407697.813471,-7.207852,-0.569139
997393,with_uv:1,5834193,33909.332142,32532,153171.073490,-34.633561,-28.522426
997394,zero_profile_user:0,13487363,69894.824760,68207,299608.316706,-27.132607,-20.651051


In [15]:
val_ll = sum(df_fm0601['loglikelihood'])
val_llr = sum(df_fm0601['loglikelihoodratio'])
n_groups = df_fm0601['group'].count()
prob_ll = math.exp(val_ll/n_groups)
prob_llr = math.exp(val_llr/n_groups)
print(val_ll, val_llr)
print(prob_ll, prob_llr)

-90383.05445889843 -52870.421178413744
0.9133656128066276 0.9483719937775533
